In [ ]:
import os, sys, torch,random

import numpy as np
from matplotlib import pyplot as plt

from scripts.utils.imutils import im_to_numpy
import scripts.models as models
import scripts.datasets as datasets
import torchvision

from skimage.measure import compare_mse as mse
from skimage.measure import compare_psnr as psnr
from tqdm import tqdm

%matplotlib inline

class objectview(object):
    def __init__(self, *args, **kwargs):
        d = dict(*args, **kwargs)
        self.__dict__ = d

method = '../checkpoint/normal_rasc_HCOCO_res_s2am_HCOCO_rascv2'
dataset_name=  method.split('_')[-2]
dataroot = '../../../Datasets/Harmonization/'

data_config  = objectview({'input_size':256,
                            'limited_dataset':0,
                            'normalized_input':False,
                            'data_augumentation':False,
                            'base_dir':dataroot,
                            'data':dataset_name})

val_loader = torch.utils.data.DataLoader(datasets.BIH('val',config=data_config,sample=[],gan_norm=False),
                                                     batch_size=1, shuffle=False,
                                                     num_workers=0, pin_memory=False)

model = models.__dict__[method.split('_')[-1]]().cuda()
model.load_state_dict(torch.load('%s/%s'%(method,'/model_best.pth.tar'))['state_dict'])
model.eval()

mse_scores,psnr_scores = [],[]
        
with torch.no_grad():
    for i, batches in enumerate(val_loader):
        
        if i% 100 ==0:
            print(i)
        
        inputs = batches['image'].cuda()
        target = batches['target'].cuda()
        mask = batches['mask'].cuda()
            
        outputs = model(torch.cat([inputs,mask],dim=1))
        
        if 'res' in method:
            outputs = outputs + inputs
        
        outputs = inputs * (1-mask) + outputs * mask    
            
        output = im_to_numpy(torch.clamp(outputs[0]*255,min=0.0,max=255.0)).astype(np.uint8)
        target = im_to_numpy(torch.clamp(batches['target'][0]*255,min=0.0,max=255.0)).astype(np.uint8)

        mse_score = mse(output,target)
        psnr_score = psnr(target,output,data_range=output.max() - output.min())
        mse_scores.append(mse_score)
        psnr_scores.append(psnr_score)
                
print(np.mean(mse_scores), np.mean(psnr_scores))

total Dataset of HCOCO is :  4283
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700


tensor(1., device='cuda:0') tensor(0., device='cuda:0')
